In [1]:
#Model 1 draft
#L1 regularization
import numpy as np
import pickle

class LassoRegression:
    def __init__(self, learning_rate=0.01, alpha=1.0,max_iter=1000 ): #alpha is the threshold strength and max_iter is number of times you can descend the slope
        
        self.learning_rate = learning_rate #intent to use gradient descent to minimize error in the linear regression model
        self.feature_names = None
        self.selected_features = {}
        self.max_iter = max_iter
        self.alpha = alpha 
        self.weights = None
        self.bias = None
        
    def _soft_threshold(self, x, thresh): #thresh = learning_rate*regularization_strength which is alpha
        # Handles the non-differentiability at 0
        return np.sign(x) * np.maximum(np.abs(x) - thresh, 0) #if weight is small set to 0, if weight is large shrink towards 0

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Initialize coefficients
        self.weights = np.zeros(n_features)   # start with w=0 
        self.bias = 0                         # intercept

        # Gradient Descent Loop
        for i in range(self.max_iter):
            # Prediction get dot product of the features with weights and include self bias
            #Formula Xw+b where X is a nx d matrix (n samples d features), and w is a dx1 matrix(d weights)
            y_pred = np.dot(X, self.weights) + self.bias 

            #Compute Residual (errors), gradient of error
            residuals = y_pred - y

            #Compute gradients, which is partial derivatives , wrt to weights and bias respectively
            dw = (1/n_samples) * np.dot(X.T, residuals) #
            db = (1/n_samples) * np.sum(residuals) #

            # Gradient descent update with shrinkage
            self.weights = self._soft_threshold(
                self.weights - self.learning_rate * dw,
                self.alpha * self.learning_rate
            )
            self.bias -= self.learning_rate * db   # bias not penalized
        return self   

    def predict(self, X):
        if self.weights is None:
            raise ValueError("Model must be fitted before making predictions")
        return np.dot(X, self.weights) + self.bias

    def get_params(self):
        """Return learned coefficients and intercept"""
        return {"weights": self.weights, "bias": self.bias}


    def get_selected_features(self, feature_names=None, threshold=1e-5):
        if self.weights is None:
            raise ValueError("Model must be fitted first")
        
        selected_idx = np.abs(self.weights) > threshold
        selected_weights = self.weights[selected_idx]
        
        if feature_names is not None:
            selected_names = [feature_names[i] for i in np.where(selected_idx)[0]]
            return dict(zip(selected_names, selected_weights))
        else:
            selected_indices = np.where(selected_idx)[0]
            return dict(zip(selected_indices, selected_weights))

def save_model(model, filename):
    #Save the trained model to a pickle file.

    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model saved to {filename}")

def load_model(filename):
    #Load a trained model from a pickle file.

    with open(filename, 'rb') as f:
        model = pickle.load(f)
    print(f"Model loaded from {filename}")
    return model


    